In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader

from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

import argparse

import numpy as np

import copy
import cv2 
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


Bad key savefig.frameon in file D:\ProgramData\Anaconda3\envs\tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file D:\ProgramData\Anaconda3\envs\tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file D:\ProgramData\Anaconda3\envs\tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr')
You probably need to get an 

In [2]:
opts = argparse.Namespace()
opts.image_channel = 6
opts.device = 'cuda'
opts.batch_size = 4
opts.imsize = 128
opts.lr = 1e-3
opts.dtype = torch.FloatTensor
opts.use_cuda = torch.cuda.is_available()

In [3]:
# use_cuda = torch.cuda.is_available()
# dtype = torch.FloatTensor
# # desired size of the output image
# imsize = 128 if use_cuda else 128  # use small size if no gpu
# device = 'cpu'
# if use_cuda:
#     device = 'cuda'

loader = transforms.Compose([
    transforms.Scale([opts.imsize, opts.imsize]),  # scale imported image
    transforms.ToTensor()])  # transform it into a torch tensor


def image_loader(image_name):
    image = Image.open(image_name)
    image = Variable(loader(image))
    # fake batch dimension required to fit network's input dimensions
    image = image.unsqueeze(0)
    return image

unloader = transforms.ToPILImage()  # reconvert into PIL image
def imshow(tensor, title=None):
    image = tensor.clone().cpu()  # we clone the tensor to not do changes on it
    image = image.view(3, opts.imsize, opts.imsize)  # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated
    
def image_unloader(tensor):
    image = tensor.clone().cpu()  # we clone the tensor to not do changes on it
    image = image.view(3, opts.imsize, opts.imsize)  # remove the fake batch dimension
    image = unloader(image)
    return image

D:\ProgramData\Anaconda3\envs\tf\lib\site-packages\torchvision\transforms\transforms.py:210: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [4]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class DenoiseNet2(nn.Module):
    def __init__(self, block):
        super(DenoiseNet2, self).__init__()
        self.e1 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(opts.image_channel, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )# 64,32,32
        self.e2 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )# 128,32,32
        self.l1 = self.make_block(block, 128, 128)
        self.l2 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(128, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )# 64,32,32
        self.l3 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(128 + 64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )# 64,32,32
        self.l4 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(128 + 64 + 64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )# 64,32,32
        self.l5 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(128 + 64 + 64 + 64, 64, 1, 1, 0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )# 1, 128, 128
        self.l6 = self.make_block(block, 128 + 64, 128 + 64)
        self.l7 = self.make_block(block, 128 + 64, 128 + 64)
        self.l8 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(128 + 64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        )# 64,32,32
        self.l9 = nn.Sequential(
            nn.ConvTranspose2d(128 + 128, 64, 3, 2 , 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )# 64, 64, 64
        self.l10 = nn.Sequential(
            # param [input_c, output_c, kernel_size, stride, padding]
            nn.Conv2d(64, 3, 1, 1, 0),
            nn.Sigmoid()
        )# 1, 64, 64
    
    def make_block(self, block, in_c, out_c):
        layers = []
        layers.append(block(in_c, out_c, 1, False))
        return nn.Sequential(*layers)

    
    def forward(self, x):
        e1 = self.e1(x)
        e2 = self.e2(e1)
        l1 = self.l1(e2)
        l2 = self.l2(l1)
        in_l3 = torch.cat((l1, l2), dim = 1)
        l3 = self.l3(in_l3)
        in_l4 = torch.cat((l1, l2, l3), dim = 1)
        l4 = self.l4(in_l4)
        in_l5 = torch.cat((l1, l2, l3, l4), dim = 1)
        l5 = self.l5(in_l5)
        in_l6 = torch.cat((l1, l5), dim = 1)
        l6 = self.l6(in_l6)
        l7 = self.l7(l6)
        l8 = self.l8(l7)
        in_l9 = torch.cat((e2, l8), dim = 1)
        l9 = self.l9(in_l9)
        l10 = self.l10(l9)
        return l10

In [5]:
net = DenoiseNet2(ResidualBlock).to(opts.device)
optimizer = optim.Adam(net.parameters(), lr=opts.lr)
criterion = nn.MSELoss()

In [6]:
IMAGE1_PATH = './train_x1'
IMAGE2_PATH = './train_x2'
TARGET_PATH = './content.jpg'
image_pathes = os.listdir(IMAGE1_PATH)
images1 = []
for img_path in image_pathes:
    if os.path.isdir(img_path):
        continue
    tmp = image_loader(os.path.join(IMAGE1_PATH,img_path)).cpu().detach().numpy()
    images1.append(tmp)
images2 = []
for img_path in image_pathes:
    if os.path.isdir(img_path):
        continue
    tmp = image_loader(os.path.join(IMAGE2_PATH,img_path)).cpu().detach().numpy()
    images2.append(tmp)
images1 = np.asarray(images1).squeeze(1)
images2 = np.asarray(images2).squeeze(1)
target = image_loader(TARGET_PATH).cpu().detach().numpy()
targets = [target for i in range(len(images1))]
targets = np.asarray(targets).squeeze(1)
train_x1 = torch.from_numpy(images1).type(opts.dtype)
train_x2 = torch.from_numpy(images2).type(opts.dtype)
train_y = torch.from_numpy(targets).type(opts.dtype)
train_dataset = TensorDataset(train_x1, train_x2, train_y)
train_loader = DataLoader(train_dataset, batch_size=opts.batch_size, shuffle=True)

In [7]:
IMAGE1_PATH = './test_x1'
IMAGE2_PATH = './test_x2'
TARGET_PATH = './content.jpg'
image_pathes = os.listdir(IMAGE1_PATH)
images1 = []
for img_path in image_pathes:
    if os.path.isdir(img_path):
        continue
    tmp = image_loader(os.path.join(IMAGE1_PATH,img_path)).cpu().detach().numpy()
    images1.append(tmp)
images2 = []
for img_path in image_pathes:
    if os.path.isdir(img_path):
        continue
    tmp = image_loader(os.path.join(IMAGE2_PATH,img_path)).cpu().detach().numpy()
    images2.append(tmp)
images1 = np.asarray(images1).squeeze(1)
images2 = np.asarray(images2).squeeze(1)
target = image_loader(TARGET_PATH).cpu().detach().numpy()
targets = [target for i in range(len(images1))]
targets = np.asarray(targets).squeeze(1)
test_x1 = torch.from_numpy(images1).type(opts.dtype)
test_x2 = torch.from_numpy(images2).type(opts.dtype)
test_y = torch.from_numpy(targets).type(opts.dtype)
test_dataset = TensorDataset(train_x1, train_x2, train_y)
test_loader = DataLoader(train_dataset, batch_size=opts.batch_size, shuffle=True)

In [10]:
for epoch in range(5):
    train_loss = 0
    for i, data in enumerate(train_loader):
        x1, x2, y = data
        x = torch.cat([x1, x2], 1)
        x = x.to(opts.device)
        y = y.to(opts.device)
        pred = net(x)
        
        optimizer.zero_grad()

        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()

    print('[Epoch %d] loss: %.8f' % (epoch + 1, loss.item()))

In [11]:
# torch.save(net.state_dict(),'net.pth')
# net.load_state_dict(torch.load('net.pth'))

<All keys matched successfully>

In [13]:
IMAGE1_PATH = './test_x1'
IMAGE2_PATH = './test_x2'
SAVE_PATH = './res'
if not os.path.exists(SAVE_PATH):
    os.mkdir(SAVE_PATH)
image_pathes = os.listdir(IMAGE1_PATH)
images = []
for img_path in image_pathes:
    if os.path.isdir(img_path):
        continue
    x1 = image_loader(os.path.join(IMAGE1_PATH,img_path)).detach()
    x2 = image_loader(os.path.join(IMAGE2_PATH,img_path)).detach()
    x = torch.cat([x1, x2], 1).to(opts.device)
    pred = net(x)
    res = image_unloader(pred)
    res.save(os.path.join(SAVE_PATH,img_path))